In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#data=pd.read_csv('/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv')

data=pd.read_csv('/kaggle/input/coviddata/full_data.csv')

In [ ]:
dataset=data[data["location"]=="India"][["date","total_deaths","new_deaths"]]

In [ ]:
dataset.shape

In [ ]:
max(dataset["date"])

In [ ]:
import matplotlib.pyplot as plt   
from sklearn.model_selection import train_test_split 
from sklearn import metrics

In [ ]:
X = np.arange(261)
X = X.reshape(-1,1)
y = dataset.dropna().iloc[:,-1].values.astype(float)


In [ ]:
# create the daily number of cases series
y = np.diff(y)
y = y.reshape(-1,1)
# create train and test dataset
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.40,random_state=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
scx = StandardScaler()
scy = StandardScaler()
x_train = scx.fit_transform(x_train)
x_test = scx.transform(x_test)
y_train = scy.fit_transform(y_train)
y_test = scy.transform(y_test)

In [ ]:
from sklearn.svm import SVR
regressor = SVR(kernel='rbf', epsilon=0.1,)
regressor.fit(x_train,y_train)
y_pred = regressor.predict(x_test)

In [ ]:
mse = metrics.mean_squared_error(y_test,y_pred)
rmse = np.sqrt(mse)

print(mse)
print(rmse)
print(regressor.score(x_test,y_test))

y_pred = np.array(scy.inverse_transform(y_pred))
y_pred = y_pred.reshape(-1,1)
x_test = np.array(scx.inverse_transform(x_test))

#plot the regression fit with original data

plt.scatter(X, y, color = 'magenta', label = 'Original Data')
plt.scatter(x_test, y_pred, color = 'green', label = 'Test Data')
plt.title('Covid19 (Support Vector Regression Model)')
plt.xlabel('Days')
plt.ylabel('Daily New Cases')
plt.legend()
plt.show()

#Predict the future values by entering number of days since onset  

y_pred = scy.inverse_transform(regressor.predict(scx.transform([[100]])))
print(y_pred)

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
dataset=np.asarray(dataset)

In [ ]:
dataset=dataset.dropna()

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
test_result=adfuller(dataset["new_deaths"])


In [ ]:
def adfuller_test(sales):
    result=adfuller(sales)
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations']
    for value,label in zip(result,labels):
        print(label+' : '+str(value) )

result=adfuller_test(dataset['new_deaths'])

In [ ]:
dataset['Sales First Difference'] = dataset['new_deaths'] - dataset['new_deaths'].shift(1)
dataset['Seasonal First Difference']=dataset['new_deaths']-dataset['new_deaths'].shift(12)
dataset.head()

In [ ]:
adfuller_test(dataset['Seasonal First Difference'].dropna())

In [ ]:
dataset['Seasonal First Difference'].plot()

In [ ]:
from pandas.plotting import autocorrelation_plot
autocorrelation_plot(dataset['new_deaths'])
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
import statsmodels.api as sm
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(dataset['Seasonal First Difference'].dropna(),lags=40,ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(dataset['Seasonal First Difference'].dropna(),lags=40,ax=ax2)

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
model=ARIMA(dataset['new_deaths'],order=(1,1,1))
model_fit=model.fit()
model_fit.summary()

In [ ]:
dataset['forecast']=model_fit.predict(start=220,end=260,dynamic=True)
dataset[['new_deaths','forecast']].plot(figsize=(12,8))

In [ ]:
import statsmodels.api as sm
model=sm.tsa.statespace.SARIMAX(dataset['new_deaths'],order=(1, 1, 1),seasonal_order=(1,1,1,12))
results=model.fit()
dataset['forecast']=results.predict(start=220,end=260,dynamic=True)
dataset[['new_deaths','forecast']].plot(figsize=(12,8))

In [ ]:
from pandas.tseries.offsets import DateOffset
future_datesseries=[pd.Timestamp(dataset["date"][-1:].values[0]) + DateOffset(days=x) for x in range(0,90)]
future_dates=[dataset.index[-1] + x for x in range(0,91)]
future_datest_df=pd.DataFrame(index=future_dates[1:],columns=dataset.columns)
future_datest_df["date"]=future_datesseries
future_datest_df.tail()

In [ ]:
future_df.tail()

In [ ]:
future_df=pd.concat([dataset,future_datest_df])

future_df['forecast'][261:] = results.predict(start = 261, end = 351, dynamic= True)
future_df[['new_deaths','forecast']].plot(figsize=(12, 8))